# This notebook will facilitate the end-to-end usage of the NordAxon Code Monkeys solution to the Adipocyte Imaging Challenge

In [34]:
import sys
sys.path.insert(0, "../../src")

### We assume that you have a folder, called "raw data dir" in this notebook, that contains raw input brightfield images, and that you want to predict nuclei, adipocytes and lipid droplets for these images. In this notebook, the end-to-end pipeline for doing this will be described.

### 1. The first step is to prepare the raw data. 
The image below shows how the pipeline looks for this. All code for this part can be found under src/prepare_training_data.py. Depending on what your file structure looks like, you will call different commands here.

What will happen in this step is that you provide two filepaths, one to the raw images, and one "middle step" where you want to store the modified input images that we will predict on. These two paths will be used in the "prepare_inference_dataset.py" file, where the following will happen:

<img src="./flows.png" alt="drawing" width="500"/>

This means that, after this script, you will have a folder where the raw input data is restructured to fit our prediction pipeline!

#### We start by defining our input and output directories for this!
 Point the input_dir to your raw data dir, and your output_dir to where you want to store the training data (Note: IMPORTANT that the raw data dir contains the directory names **20x_images, 40x_images, 60x_images** see below!)

In [40]:
# First, copy inference data to '../data/01_raw/' directory
path_to_your_data = "../data/01_raw/adipocyte_raw_data/"

!mkdir '../data/01_raw/adipocyte_inference_data'
!cp -r $path_to_your_data '../data/01_raw/adipocyte_inference_data'

mkdir: cannot create directory ‘../data/01_raw/adipocyte_inference_data’: File exists


In [41]:
# Change here:
raw_data_dir = '../data/01_raw/adipocyte_inference_data'
prediction_data_dir = "../data/07_inference_data/"

#### If your file structure corresponds to the following (target and mask folders are optional):

- input_dir
    - input
        - 20x_images
            - img1
            - img2
            - ...
        - 40x_images
            - ...
        - 60x_images
            - ...
    - target
        - 20x_images
        - 40x_images
        - 60x_images

or

- input_dir
    - 20x_images
        - input
            - img1
            - img2
            - ...
        - target
    - 40x_images
        - ...
    - 60x_images
        - ...
        
or

- input_dir
    - 20x_images
        - img1
        - img2
        - ...
    - 40x_images
        - ...
    - 60x_images
        - ...
        
#### you will run the cell directly below this one:

In [42]:
!python ../src/prepare_inference_data.py --input-dir $raw_data_dir --output-dir $prediction_data_dir

Namespace(input_dir='../data/01_raw/adipocyte_inference_data', output_dir='../data/07_inference_data/')
Found 2242 .tif images in total
100%|███████████████████████████████████████| 2242/2242 [08:22<00:00,  4.46it/s]


### 2. The second step is to predict for each target
When predicting, the pipeline differs slightly depending on the target image that we want to predict. This is because we have chosen to use masks for the A01 target (nuclei).
In the image below, the flow is described for prediction. In the code, predictions.py will be called one time for each magnification.

<img src="./flows2.png" alt="drawing" width="600"/>

What this flow chart shows is that the masks are only loaded if the target is the nuclei, i.e. A01.

Now, we need to define an output path to where we want our predicted images to be! And, we want to define the path to the model weights that we want to use. Fortunately, we have done this for you, but you are free to change this if you wish.

In [1]:
weights_path_A1_mask = "../data/05_saved_models/normalized_bias/A1_segmentation_model_2.pth"

In [2]:
input_dir = "../data/07_inference_data/input/20x_images"
output_dir = "../data/07_inference_data/masks/20x_images"
!python ../src/predict_segmentation.py --input-dir $input_dir --output-dir $output_dir --weights_path $weights_path_A1_mask

input_dir = "../data/07_inference_data/input/40x_images"
output_dir = "../data/07_inference_data/masks/40x_images"
!python ../src/predict_segmentation.py --input-dir $input_dir --output-dir $output_dir --weights_path $weights_path_A1_mask

input_dir = "../data/07_inference_data/input/60x_images"
output_dir = "../data/07_inference_data/masks/60x_images"
!python ../src/predict_segmentation.py --input-dir $input_dir --output-dir $output_dir --weights_path $weights_path_A1_mask

Namespace(crop_size=1024, input_dir='../data/07_inference_data/input/20x_images', output_dir='../data/07_inference_data/masks/20x_images', stride=512, v2=True, weights_path='../data/05_saved_models/normalized_bias/A1_segmentation_model_2.pth')
 79%|██████████████████████████████████         | 38/48 [04:08<01:11,  7.20s/it]Successfully saved to ../data/07_inference_data/masks/20x_images/AssayPlate_Greiner_#655090_D04_T0001F005L01A01Z01C01.tif


100%|███████████████████████████████████████████| 48/48 [05:26<00:00,  6.80s/it]
Namespace(crop_size=1024, input_dir='../data/07_inference_data/input/40x_images', output_dir='../data/07_inference_data/masks/40x_images', stride=512, v2=True, weights_path='../data/05_saved_models/normalized_bias/A1_segmentation_model_2.pth')
 45%|███████████████████▍                       | 29/64 [03:43<04:18,  7.40s/it]Successfully saved to ../data/07_inference_data/masks/40x_images/AssayPlate_Greiner_#655090_D04_T0001F007L01A01Z01C01.tif


100%|███████████████████████████████████████████| 64/64 [08:04<00:00,  7.58s/it]
Namespace(crop_size=1024, input_dir='../data/07_inference_data/input/60x_images', output_dir='../data/07_inference_data/masks/60x_images', stride=512, v2=True, weights_path='../data/05_saved_models/normalized_bias/A1_segmentation_model_2.pth')
672
96
  4%|█▊                                          | 4/96 [00:33<13:14,  8.63s/it]Successfully saved to ../data/07_inference_data/masks/60x_images/AssayPlate_Greiner_#655090_D03_T0001F011L01A01Z01C01.tif


 47%|████████████████████▏                      | 45/96 [05:37<06:06,  7.18s/it]Successfully saved to ../data/07_inference_data/masks/60x_images/AssayPlate_Greiner_#655090_C03_T0001F001L01A01Z01C01.tif


 90%|██████████████████████████████████████▌    | 86/96 [11:03<01:30,  9.06s/it]Successfully saved to ../data/07_inference_data/masks/60x_images/AssayPlate_Greiner_#655090_D02_T0001F003L01A01Z01C01.tif


100%|███████████████████████████████████████████| 96/96 [12:24<00:00,  7.76s/it]


In [ ]:
#weights_path_A1 = "../../data/05_saved_models/normalized_bias/A1_g_best.pth" # Without mask input
weights_path_A1 = "../../data/05_saved_models/normalized_bias/A1_g_best_wmask.pth" 

input_dir = "../data/07_inference_data/input/20x_images"
mask_dir = "../data/07_inference_data/masks/20x_images"
output_dir = "../data/04_generated_images/inference_results/20x_images"
!python ../src/predict.py --mask --mask-dir $mask_dir --target A1 --input-dir $input_dir  --output-dir $output_dir --weights $weights_path_A1 --mag 20x --match-histogram

input_dir = "../data/07_inference_data/input/40x_images"
mask_dir = "../data/07_inference_data/masks/40x_images"
output_dir = "../data/04_generated_images/inference_results/40x_images"
!python ../src/predict.py --mask --mask-dir $mask_dir --target A1 --input-dir $input_dir  --output-dir $output_dir --weights $weights_path_A1 --mag 40x --match-histogram

input_dir = "../data/07_inference_data/input/60x_images"
mask_dir = "../data/07_inference_data/masks/60x_images"
output_dir = "../data/04_generated_images/inference_results/60x_images"
!python ../src/predict.py --mask --mask-dir $mask_dir --target A1 --input-dir $input_dir  --output-dir $output_dir --weights $weights_path_A1 --mag 60x --match-histogram


In [ ]:
weights_path_A2 = "../../data/05_saved_models/normalized_bias/A2_g_best.pth"

input_dir = "../data/07_inference_data/input/20x_images"
output_dir = "../data/04_generated_images/inference_results/20x_images"
!python ../src/predict.py --target A2 --input-dir $input_dir  --output-dir $output_dir --weights $weights_path_A2 --mag 20x --match-histogram

input_dir = "../data/07_inference_data/input/40x_images"
output_dir = "../data/04_generated_images/inference_results/40x_images"
!python ../src/predict.py --target A2 --input-dir $input_dir  --output-dir $output_dir --weights $weights_path_A2 --mag 40x --match-histogram

input_dir = "../data/07_inference_data/input/60x_images"
output_dir = "../data/04_generated_images/inference_results/60x_images"
!python ../src/predict.py --target A2 --input-dir $input_dir  --output-dir $output_dir --weights $weights_path_A2 --mag 60x --match-histogram


In [ ]:
weights_path_A3 = "../../data/05_saved_models/normalized_bias/A3_g_best.pth"

input_dir = "../data/07_inference_data/input/20x_images"
output_dir = "../data/04_generated_images/inference_results/20x_images"
!python ../src/predict.py --target A3 --input-dir $input_dir  --output-dir $output_dir --weights $weights_path_A3 --mag 20x --match-histogram

input_dir = "../data/07_inference_data/input/40x_images"
output_dir = "../data/04_generated_images/inference_results/40x_images"
!python ../src/predict.py --target A3 --input-dir $input_dir  --output-dir $output_dir --weights $weights_path_A3 --mag 40x --match-histogram

input_dir = "../data/07_inference_data/input/60x_images"
output_dir = "../data/04_generated_images/inference_results/60x_images"
!python ../src/predict.py --target A3 --input-dir $input_dir  --output-dir $output_dir --weights $weights_path_A3 --mag 60x --match-histogram


In [ ]:
# Example of working pipeline
"""
for mag in ["20x", "40x", "60x"]:
    input_dir = f"../../data/03_training_data/normalized_bias/valid/input/{mag}_images"
    output_dir = f"../../data/04_generated_images/valid_inference_results/{mag}_images"
    !python ../../src/predict.py --target A01 --input-dir $input_dir  --output-dir $output_dir --weights $weights_path_A1 --mag {mag} --match-histogram

                       
for mag in ["20x", "40x", "60x"]:
    input_dir = f"../../data/03_training_data/normalized_bias/valid/input/{mag}_images"
    output_dir = f"../../data/04_generated_images/valid_inference_results/{mag}_images"
    !python ../../src/predict.py --target A02 --input-dir $input_dir  --output-dir $output_dir --weights $weights_path_A2 --mag {mag} --match-histogram
                       

for mag in ["20x", "40x", "60x"]:
    input_dir = f"../../data/03_training_data/normalized_bias/valid/input/{mag}_images"
    output_dir = f"../../data/04_generated_images/valid_inference_results/{mag}_images"
    !python ../../src/predict.py --target A03 --input-dir $input_dir  --output-dir $output_dir --weights $weights_path_A3 --mag {mag} --match-histogram
"""